# Machine Learning - Practical 1

This notebook provides you with the assignments and the overall code structure you need to complete the assignment. There are also questions that you need to answer in text form. Please use full sentences and reasonably correct spelling/grammar.

Regarding submission & grading:

- Work in groups of three and hand in your solution as a group.

- Solutions need to be uploaded to StudIP until the submission date indicated in the course plan. Please upload a copy of this notebook and a PDF version of it after you ran it.

- Solutions need to be presented to tutors in tutorial. Presentation dates are listed in the course plan. Every group member needs to be able to explain everything.

- You have to solve N-1 practicals to get admission to the exam.

- For plots you create yourself, all axes must be labeled. 

- Do not change the function interfaces.

In [ ]:
%matplotlib inline

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

## The  dataset

The dataset consists of over 20.000 materials and lists their physical features. From these features, we want to learn how to predict the critical temperature, i.e. the temperature we need to cool the material to so it becomes superconductive. First load and familiarize yourself with the data set a bit.

In [ ]:
#you will need to change the directory path
data = pd.read_csv('/kaggle/input/superconduct-train/superconduct_train.csv')
print(data.shape)

In [ ]:
data.head()

Because the dataset is rather large, we prepare a small subset of the data as training set, and another subset as test set. To make the computations reproducible, we set the random seed.

In [ ]:
target_clm = 'critical_temp'  # the critical temperature is our target variable
n_trainset = 200  # size of the training set
n_testset = 500  # size of the test set

In [ ]:
# set random seed to make sure every test set is the same
np.random.seed(seed=1)

idx = np.arange(data.shape[0])
idx_shuffled = np.random.permutation(idx)  # shuffle indices to split into training and test set

test_idx = idx_shuffled[:n_testset]
train_idx = idx_shuffled[n_testset:n_testset+n_trainset]
train_full_idx = idx_shuffled[n_testset:]

X_test = data.loc[test_idx, data.columns != target_clm].values
y_test = data.loc[test_idx, data.columns == target_clm].values
print('Test set shapes (X and y)', X_test.shape, y_test.shape)

X_train = data.loc[train_idx, data.columns != target_clm].values
y_train = data.loc[train_idx, data.columns == target_clm].values
print('Small training set shapes (X and y):', X_train.shape, y_train.shape)

X_train_full = data.loc[train_full_idx, data.columns != target_clm].values
y_train_full = data.loc[train_full_idx, data.columns == target_clm].values
print('Full training set shapes (X and y):', X_train_full.shape, y_train_full.shape)

## Task 1: Plot the dataset

To explore the dataset, use `X_train_full` and `y_train_full` for two descriptive plots:

* **Histogram** of the target variable. Use `plt.hist`.

* **Scatterplots** relating the target variable to one of the feature values. For this you will need 81 scatterplots. Arrange them in one big figure with 9x9 subplots. Use `plt.scatter`. You may need to adjust the marker size and the alpha blending value. 

Furthermore, we need to normalize the data, such that each feature has a mean of zero mean and a variance of one. Implement a function `normalize` which normalizes the data. Print the means and standard variation of the first five features before and after. For simplicity we will normalize train and test set independently. It is better practice to normalize the test set with mean and variance of the test set. Maybe you can think of situations / reasons where this makes a difference.

In [ ]:
# Histogram of the target variable

#minimum = min(y_train_full)
#maximum = max(y_train_full)

num_bins = 10
n, bins, patches = plt.hist(y_train_full, bins = num_bins)
plt.xlabel('temperature')
plt.ylabel('materials')

In [ ]:
# Scatter plots of the target variable vs. features

size, no_features = X_train_full.shape

f = plt.figure()
f, axes = plt.subplots(nrows = 9, ncols = 9, figsize=(15, 15))


for i in range(no_features):
    ax1 = int(i/9)
    ax2 = i%9
    axes[ax1][ax2].scatter(X_train_full[:,i],y_train_full, marker = ".")
    #axes[ax1,ax2].set_xlabel('Feature {}'.format(i))
    #axes[ax1,ax2].set_ylabel('Target Variable')

plt.show()

In [ ]:
# Normalize

def normalize(X):
    
    mean = np.mean(X, axis = 0)
    std = np.std(X, axis = 0)
    
    print('Mean:', X[:, :5].mean(axis=0))
    print('Std:', X[:, :5].std(axis=0))
    
    return (X - mean)/std


X_train_full_norm = normalize(X_train_full)
X_train_norm = normalize(X_train)
X_test_norm = normalize(X_test)


Which material properties may be useful for predicting superconductivity? What other observations can you make?

 YOUR ANSWER HERE
 
 None of them are really good as they dont look like functions on their own we need a linear combination of them.

## Task 2:  Implement your own OLS estimator

We want to use linear regression to predict the critical temperature. Implement the ordinary least squares estimator without regularization 'by hand':

$w = (X^TX)^{-1}X^Ty$

To make life a bit easier, we provide a function that can be used to plot regression results. In addition it computes the mean squared error and the squared correlation between the true and predicted values. 

In [ ]:
def plot_regression_results(y_test, y_pred, weights):
    '''Produces three plots to analyze the results of linear regression:
        -True vs predicted
        -Raw residual histogram
        -Weight histogram

    Inputs:
        y_test: (n_observations,) numpy array with true values
        y_pred: (n_observations,) numpy array with predicted values
        weights: (n_weights) numpy array with regression weights'''

    print('MSE: ', mean_squared_error(y_test, y_pred))
    print('r^2: ', r2_score(y_test, y_pred))

    fig, ax = plt.subplots(1, 3, figsize=(9, 3))
    # predicted vs true
    ax[0].scatter(y_test, y_pred)
    ax[0].set_title('True vs. Predicted')
    ax[0].set_xlabel('True %s' % (target_clm))
    ax[0].set_ylabel('Predicted %s' % (target_clm))

    # residuals
    error = np.squeeze(np.array(y_test)) - np.squeeze(np.array(y_pred))
    ax[1].hist(np.array(error), bins=30)
    ax[1].set_title('Raw residuals')
    ax[1].set_xlabel('(true-predicted)')

    # weight histogram
    ax[2].hist(weights, bins=30)
    ax[2].set_title('weight histogram')

    plt.tight_layout()

As an example, we here show you how to use this function with random data. 

In [ ]:
# weights is a vector of length 82: the first value is the intercept (beta0), then 81 coefficients
weights = np.random.randn(82)

# Model predictions on the test set
y_pred_test = np.random.randn(y_test.size)

plot_regression_results(y_test, y_pred_test, weights)

Implement OLS linear regression yourself. Use `X_train` and `y_train` for estimating the weights and compute the MSE and $r^2$ from `X_test`. When you call our plotting function with the regession result, you should get mean squared error of 707.8.

In [ ]:
def OLS_regression(X_test, X_train, y_train):
    '''Computes OLS weights for linear regression without regularization on the training set and
       returns weights and testset predictions.

       Inputs:
         X_test: (n_observations, 81), numpy array with predictor values of the test set
         X_train: (n_observations, 81), numpy array with predictor values of the training set
         y_train: (n_observations,) numpy array with true target values for the training set

       Outputs:
         weights: The weight vector for the regerssion model including the offset
         y_pred: The predictions on the TEST set

       Note:
         Both the training and the test set need to be appended manually by a columns of 1s to add
         an offset term to the linear regression model.
    '''

    # ---------------- INSERT CODE ----------------------
    
    X_train = np.concatenate([np.ones((X_train.shape[0],1)), X_train], axis=1)
    X_test = np.concatenate([np.ones((X_test.shape[0],1)), X_test], axis=1)
    
    weights = np.linalg.inv(X_train.T@X_train)@X_train.T@y_train
    y_pred = X_test@weights

    # ---------------- END CODE -------------------------

    return weights, y_pred

In [ ]:
# Plots of the results

weights, y_pred = OLS_regression(X_test_norm, X_train_norm, y_train)
plot_regression_results(y_test, y_pred, weights)


What do you observe? Is the linear regression model good?

YOUR ANSWER HERE
Although you cannot go below the critical temperature like the model does in the predictions, the results are decent imagining a line of x=y starting from the origin.

## Task 3: Compare your implementation to sklearn

Now, familarize yourself with the sklearn library. In the section on linear models:

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model

you will find `sklearn.linear_model.LinearRegression`, the `sklearn` implementation of the OLS estimator. Use this sklearn class to implement OLS linear regression. Again obtain estimates of the weights on `X_train` and `y_train` and compute the MSE and $r^2$ on `X_test`.


In [ ]:
def sklearn_regression(X_test, X_train, y_train):
    '''Computes OLS weights for linear regression without regularization using the sklearn library on the training set and
       returns weights and testset predictions.

       Inputs:
         X_test: (n_observations, 81), numpy array with predictor values of the test set
         X_train: (n_observations, 81), numpy array with predictor values of the training set
         y_train: (n_observations,) numpy array with true target values for the training set

       Outputs:
         weights: The weight vector for the regerssion model including the offset
         y_pred: The predictions on the TEST set

       Note:
         The sklearn library automatically takes care of adding a column for the offset.
    '''

    # ---------------- INSERT CODE ----------------------
    model = linear_model.LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    weights = model.coef_

    # ---------------- END CODE -------------------------

    return weights, y_pred

In [ ]:
weights, y_pred = sklearn_regression(X_test_norm, X_train_norm, y_train)
plot_regression_results(y_test, y_pred, weights)

If you implemented everything correctly, the MSE is again 707.8.

Fit the model using the larger training set, `X_train_full` and `y_train_full`, and again evaluate on `X_test`.

In [ ]:
weights, y_pred = sklearn_regression(X_test_norm, X_train_full_norm, y_train_full)
plot_regression_results(y_test, y_pred, weights)

 How does test set performance change? What else changes?

YOU ANSWER HERE

It's better, it is more linear, less scattered in the plot at the left (should be an x=y line) and we can see that MSE error is much smaller.

## Task 4: Regularization with ridge regression

We will now explore how a penalty term on the weights can improve the prediction quality for finite data sets. Implement the analytical solution of ridge regression 

$w = (X^TX + \alpha I_D)^{-1}X^Ty$


as a function that can take different values of $\alpha$, the regularization strength, as an input. In the lecture, this parameter was called $\lambda$, but this is a reserved keyword in Python.

In [ ]:
def ridge_regression(X_test, X_train, y_train, alpha):
    '''Computes OLS weights for regularized linear regression with regularization strength alpha
       on the training set and returns weights and testset predictions.

       Inputs:
         X_test: (n_observations, 81), numpy array with predictor values of the test set
         X_train: (n_observations, 81), numpy array with predictor values of the training set
         y_train: (n_observations,) numpy array with true target values for the training set
         alpha: scalar, regularization strength

       Outputs:
         weights: The weight vector for the regression model including the offset
         y_pred: The predictions on the TEST set

       Note:
         Both the training and the test set need to be appended manually by a columns of 1s to add
         an offset term to the linear regression model.
    '''

    # ---------------- INSERT CODE ----------------------

    X_train = np.concatenate([np.ones((X_train.shape[0],1)), X_train], axis=1)
    X_test = np.concatenate([np.ones((X_test.shape[0],1)), X_test], axis=1)
    
    weights = np.linalg.inv(X_train.T@X_train + alpha*np.identity(X_train.shape[1]))@X_train.T@y_train
    y_pred = X_test@weights

    # ---------------- END CODE -------------------------

    return weights, y_pred

Run the ridge regression on `X_train` with an alpha value of 10 and plot the obtained weights.

In [ ]:
# Run ridge regression with alpha=10
weights, y_pred = ridge_regression(X_test_norm, X_train_norm, y_train, alpha = 10)

# Plot regression results
plot_regression_results(y_test, y_pred, weights)


Now test a range of log-spaced $\alpha$s (~10-20), which cover several orders of magnitude, e.g. from 10^-7 to 10^7. 

* For each $\alpha$, you will get one model with one set of weights. 
* For each model, compute the error on the test set. 

Store both the errors and weights of all models for later use. You can use the function `mean_squared_error` from sklearn (imported above) to compute the MSE.


In [ ]:
alphas = np.logspace(-7, 7, 100)

# ---------------- INSERT CODE ----------------------

mse = []
W = weights

for alpha in alphas:
    weights, y_pred = ridge_regression(X_test_norm, X_train_norm, y_train, alpha)
    mse.append(mean_squared_error(y_test, y_pred))   
    W = np.concatenate([W, weights], axis=1)

W = W[:,1:]
# ---------------- END CODE -------------------------

print(W.shape)

Make a single plot that shows for each coefficient how it changes with $\alpha$, i.e. one line per coefficient. Also think about which scale is appropriate for your $\alpha$-axis. You can set this using `plt.xscale(...)`.

In [ ]:
# Plot of coefficients vs. alphas

for weight in W:
    
    plt.xscale('log') 
    plt.plot(alphas, weight) 
    plt.xlabel('alphas')
    plt.ylabel('weights')

plt.show()



Why are the values of the weights largest on the left? Do they all change monotonically? 

YOUR ANSWER HERE

As the magnitude of the penalty term increases via alpha, the more emphasis is put on to reducing the magnitude of the weights.

Plot how the performance (i.e. the error) changes as a function of $\alpha$.

In [ ]:
# Plot of MSE  vs. alphas

plt.plot(alphas, mse)
plt.xscale('log')
plt.xlabel('alphas')
plt.ylabel('mse')
plt.show()

Which value of $\alpha$ gives the minimum MSE? Is it better than the unregularized model? Why should the curve reach ~700 on the left?

YOUR ANSWER HERE
Around 10 gives the minimum. The mse is lower than before. Yes it should because as penalty term goes to 0, the ridge regression becomes regular linear regression with only MSE loss.

Now implement the same model using sklearn. Use the `linear_model.Ridge` object to do so.


In [ ]:
def ridge_regression_sklearn(X_test, X_train, y_train, alpha):
    '''Computes OLS weights for regularized linear regression with regularization strength alpha using the sklearn
       library on the training set and returns weights and testset predictions.

       Inputs:
         X_test: (n_observations, 81), numpy array with predictor values of the test set
         X_train: (n_observations, 81), numpy array with predictor values of the training set
         y_train: (n_observations,) numpy array with true target values for the training set
         alpha: scalar, regularization strength

       Outputs:
         weights: The weight vector for the regerssion model including the offset
         y_pred: The predictions on the TEST set

       Note:
         The sklearn library automatically takes care of adding a column for the offset.
    '''

    # ---------------- INSERT CODE ----------------------
    model = linear_model.Ridge(alpha = alpha)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    weights = model.coef_


    # ---------------- END CODE -------------------------

    return weights, y_pred

This time, only plot how the performance changes as a function of $\alpha$. 

In [ ]:
# Plot of MSE  vs. alphas

def findplotMSE(X_test, X_train, y_train, y_test, alphas):
    
    mse = []

    for alpha in alphas:
        weights, y_pred = ridge_regression_sklearn(X_test, X_train, y_train, alpha)
        mse.append(mean_squared_error(y_test, y_pred))   
    
    return mse


findplotMSE(X_test_norm, X_train_norm, y_train, y_test, alphas)
plt.plot(alphas, mse)
plt.xscale('log')
plt.show()

Note: Don't worry if the curve is not exactly identical to the one you got above. The loss function we wrote down in the lecture  has $\alpha$ defined a bit differently compared to sklearn. However, qualitatively it should look the same.

## Task 5: Cross-validation

Until now, we always estimated the error on the test set directly. However, we typically do not want to tune hyperparameters of our inference algorithms like $\alpha$ on the test set, as this may lead to overfitting. Therefore, we tune them on the training set using cross-validation. As discussed in the lecture, the training data is here split in `n_folds`-ways, where each of the folds serves as a held-out dataset in turn and the model is always trained on the remaining data. Implement a function that performs cross-validation for the ridge regression parameter $\alpha$. You can reuse functions written above.

In [ ]:
def ridgeCV(X, y, n_folds, alphas):
    '''Runs a n_fold-crossvalidation over the ridge regression parameter alpha.
       The function should train the linear regression model for each fold on all values of alpha.

      Inputs:
        X: (n_obs, n_features) numpy array - predictor
        y: (n_obs,) numpy array - target
        n_folds: integer - number of CV folds
        alphas: (n_parameters,) - regularization strength parameters to CV over

      Outputs:
        cv_results_mse: (n_folds, len(alphas)) numpy array, MSE for each cross-validation fold

      Note:
        Fix the seed for reproducibility.
    '''

    cv_results_mse = np.zeros((n_folds, len(alphas)))
    np.random.seed(seed=2)

    # ---------------- INSERT CODE ----------------------
    
    step_size = int(X.shape[0]/n_folds)
    
    for i in range(n_folds):
        X_test = X[i*step_size:(i+1)*step_size]
        y_test = y[i*step_size:(i+1)*step_size]
        X_train = np.delete(X, np.s_[i*step_size:(i+1)*step_size], axis=0)
        y_train = np.delete(y, np.s_[i*step_size:(i+1)*step_size], axis=0)
        
        mse = findplotMSE(X_test, X_train, y_train,y_test, alphas)
        cv_results_mse[i] = mse


    # ---------------- END CODE -------------------------

    return cv_results_mse

Now we run 10-fold cross-validation using the training data of a range of $\alpha$s.

In [ ]:
alphas = np.logspace(-7, 7, 100)
mse_cv = ridgeCV(X_train_norm, y_train, n_folds=10, alphas=alphas)

We plot the MSE trace for each fold separately:

In [ ]:
plt.plot(alphas, mse_cv.T, '.-')
plt.xscale('log')
plt.xlabel('alpha')
plt.ylabel('Mean squared error')
plt.tight_layout()

We also plot the average across folds:

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(alphas, np.mean(mse_cv, axis=0), '.-')
plt.xscale('log')
plt.xlabel('alpha')
plt.ylabel('Mean squared error')
plt.tight_layout()

What is the optimal $\alpha$? Is it similar to the one found on the test set? Do the cross-validation MSE and the test-set MSE match well or differ strongly?

YOUR ANSWER HERE
Again the optimal alpha is around 10, it is nearly the same as on the test set, they match very well cross validation should definitely be used.

We will now run cross-validation on the full training data. This will take a moment, depending on the speed of your computer. Afterwards, we will again plot the mean CV curves for the full data set (blue) and the small data set (orange).

In [ ]:
alphas = np.logspace(-7, 7, 100)
mse_cv_full = ridgeCV(X_train_full_norm, y_train_full, n_folds=10, alphas=alphas)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(alphas, np.mean(mse_cv_full, axis=0), '.-')
plt.plot(alphas, np.mean(mse_cv, axis=0), '.-')
plt.xscale('log')
plt.xlabel('alpha')
plt.ylabel('Mean squared error')
plt.tight_layout()

We zoom in on the blue curve to the very left:

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(alphas, np.mean(mse_cv_full, axis=0), '.-')
plt.xscale('log')
minValue = np.min(np.mean(mse_cv_full, axis=0))
plt.ylim([minValue-.01, minValue+.02])
plt.xlabel('alpha')
plt.ylabel('Mean squared error')
plt.tight_layout()

Why does the CV curve on the full data set look so different? What is the optimal value of $\alpha$ and why is it so much smaller than on the small training set?

YOUR ANSWER HERE
the optimal value is around 0.03,